# Environment Setting Up

In [1]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/Live-Air-Quality'

In [2]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 1. API Exploration

In [3]:
from AQI.utils.logger import get_logger

# Initializing the logger to test for exploration purposes
logger = get_logger("test")

In [4]:
import os
from openaq import OpenAQ
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def setup_openaq(secret_env_var: str = "OPENAQI_API_KEY") -> OpenAQ:
    """
    Load the OpenAQ API key from environment and return an authenticated OpenAQ client.

    Args:
        secret_env_var (str): Name of the environment variable holding the API key.

    Returns:
        OpenAQ: An authenticated OpenAQ client object.

    Raises:
        ValueError: If the API key is not found in the environment.
        Exception: If client creation fails for any other reason.
    """
    # Read API key from environment
    openaq_api = os.getenv(secret_env_var)

    if openaq_api is None:
        logger.error("Unable to find OpenAQ API key in environment.")
        raise ValueError(f"{secret_env_var} not found or empty.")
    
    try:
        client = OpenAQ(api_key=openaq_api)
        logger.info(f"Successfully initialized OpenAQ client.")
        return client
    
    except Exception as e:
        logger.error(f"Unexpected error while creating OpenAQ client: {e}")
        raise e

In [45]:
client = setup_openaq()
client

[2025-08-11 12:56:51,437: INFO: 3860316096: Successfully initialized OpenAQ client.]


In [16]:
for attr in dir(client):
    if not attr.startswith("_") and not attr.endswith("_"):
        print(attr)

api_key
base_url
build_request_headers
close
countries
headers
instruments
licenses
locations
manufacturers
measurements
owners
parameters
providers
resolve_headers
sensors
transport


In [63]:
# Finding stations around NYC

data = client.locations.list(
    bbox=[-74.25909, 40.477399, -73.700181, 40.917577],
    limit=100
    )
    
# Check if query works
data.meta

Meta(name='openaq-api', website='/', page=1, limit=100, found=52)

In [64]:
data.meta.name

'openaq-api'

In [65]:
len(data.results)

52

In [77]:
data.results[0].__dict__

{'id': 384,
 'name': 'CCNY',
 'locality': 'New York-Northern New Jersey-Long Island',
 'timezone': 'America/New_York',
 'country': CountryBase(id=155, code='US', name='United States'),
 'owner': OwnerBase(id=4, name='Unknown Governmental Organization'),
 'provider': ProviderBase(id=119, name='AirNow'),
 'is_mobile': False,
 'is_monitor': True,
 'instruments': [InstrumentBase(id=2, name='Government Monitor')],
 'sensors': [SensorBase(id=671, name='o3 ppm', parameter=ParameterBase(id=10, name='o3', units='ppm', display_name='O₃')),
  SensorBase(id=673, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5'))],
 'coordinates': Coordinates(latitude=40.8197, longitude=-73.9481),
 'bounds': [-73.9481, 40.8197, -73.9481, 40.8197],
 'distance': None,
 'datetime_first': Datetime(utc='2016-03-12T09:00:00Z', local='2016-03-12T04:00:00-05:00'),
 'datetime_last': Datetime(utc='2025-08-11T12:00:00Z', local='2025-08-11T08:00:00-04:00')}

In [73]:
loc_info = {}
for location in data.results:
    loc_info[location.id] = location.name

assert len(data.results) == len(loc_info)

In [76]:
loc_info

{384: 'CCNY',
 386: 'Susan Wagner',
 625: 'Manhattan/IS143',
 626: 'Bronx - IS52',
 628: 'Maspeth',
 631: 'Queens',
 642: 'PS 19',
 648: 'Bklyn - PS 314',
 662: 'Division Street',
 664: 'Bklyn - PS274',
 665: 'Bronx - IS74',
 666: 'Pfizer Lab',
 853: 'East Orange',
 857: 'Fort Lee Near Road',
 928: 'Jersey City FH',
 971: 'Elizabeth Trailer',
 974: 'Newark Firehouse',
 984: 'Leonia',
 1122: 'Jersey City',
 1496: 'Bayonne',
 2193: 'Elizabeth',
 8749: 'Queens Near-road',
 292229: 'Morrisania',
 387315: 'Union City High Scho',
 496096: 'Port Richmond',
 1236043: 'Franklin Avenue',
 1738519: 'Bayside, NY',
 1775647: 'State Dept of Environmental Conservation',
 1824516: 'Bayside, NY',
 2153518: 'Jersey City Heights',
 2386747: 'Carteret, NJ',
 2453305: 'State Dept of Environmental Conservation',
 2453313: 'State Dept of Environmental Conservation',
 2616564: '7th Ave and W 16th St',
 2903996: 'Win Son',
 3041962: 'Near Bay 50 St',
 3103622: 'Montclair, NJ',
 3145067: 'Metal fence on PS9 bac

In [81]:
from AQI.utils.common import create_directories, save_json

create_directories(["artifacts/data"])
save_json(save_path=Path("artifacts/data/sensor_locations.json"), data=loc_info)


[2025-08-11 13:22:09,381: INFO: common: Directory: artifacts/data created successfully.]
[2025-08-11 13:22:09,383: INFO: common: Directory: artifacts/data created successfully.]
[2025-08-11 13:22:09,385: INFO: common: JSON file saved at: artifacts/data/sensor_locations.json]
